<a href="https://colab.research.google.com/github/xiaoyi-yuxiao/MachineLearningTheory/blob/master/Logistic_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 pip install yfinance

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [ ]:
class ML_logistic:

  def __init__(self, X, Y):

    self.__cost = 0
    self.__theta = np.array([])#this is dynamic
    self.__num_of_feature = 1
    self.__X,self.__Y = self.handle_data(X,Y)

  def __add_theta(self):
    #add an initial theta every time when get a new variable
    self.__theta = np.append(self.__theta,0)
    self.__num_of_feature = self.__num_of_feature+1

  def add_column(self,X):
    #else if there is data, then append data
    #if the shape do not
      X = X.to_numpy()
      X_reshaped = X.reshape((len(X),1))
      X_reshaped = (X_reshaped-X_reshaped.mean())/np.std(X_reshaped)
      self.__X = np.append(self.__X,X_reshaped,axis=1)
      self.__add_theta()

  #make y to be 0 or 1
  def set_Y(self,Y):
    
    Y[Y>0] = 1
    Y[Y<0] = 0
    return Y

  def handle_data(self,X,Y):
    
    #convert pandas to numpy
    X = X.to_numpy()
    Y = Y.to_numpy()
    #reshape the data to form a matrix
    X_reshaped = X.reshape((len(X),1))
    Y_reshaped = Y.reshape((len(Y),1))
    X_reshaped = (X_reshaped-X_reshaped.mean())/np.std(X_reshaped)
    #use set_Y to set 0 1 to different cluster
    Y_reshaped = self.set_Y(Y_reshaped)

    #add ones to X_reshaped
    add_ones = np.ones([len(X),1])
    Appended_X = np.append(add_ones,X_reshaped,axis=1)
    #add theta for one's
    self.__theta = np.append(self.__theta,0)
    #add theta for other feature
    self.__add_theta()

    return Appended_X,Y_reshaped
  
  def sigmoid(self,z):
    
    sigmoid_z = 1/(1+np.exp(-1*z))
    return sigmoid_z

  #hypothesis function
  def comgput_cost(self,hypos):

    self.__cost = np.dot(-1*self.__Y.transpose(), np.log(hypos))+np.dot(-1*(1-self.__Y).transpose(),np.log(1-hypos))

  def derivative(self):

    #calculate hypothesis reshape hypothesis
    z = np.dot(self.__X, self.__theta)
    h = self.sigmoid(z)
    reshaped_h = h.reshape((len(h),1))#reshaped hypothesis

    #calculate difference between Y and hypothesis
    self.comgput_cost(reshaped_h)

    deri_theta = []
    
    #calculate derivative for the features
    for i in range(self.__num_of_feature):

      temp = reshaped_h-self.__Y
      deri_theta_i = (1/len(self.__X)) * np.dot(temp.transpose(), self.__X[:,i])
      deri_theta.append(deri_theta_i)

    return deri_theta

  def get_cost(self):

    return self.__cost

  def update(self, learning_rate):

    deri_theta= self.derivative()

    #update each theta
    for i in range(self.__num_of_feature):
      self.__theta[i] = self.__theta[i] - learning_rate * deri_theta[i]

  def gradient_descent(self, learning_rate=0.05, stop_condition=0.1):

    #interation times
    i = 1

    #interating to find minimum derivative for theta0 and theta1
    while True:

      deri_theta = self.derivative()
      print("Cost on interation" + str(i) + " is " + str(self.__cost))
      print(sum(np.abs(deri_theta)))
      
      if sum(np.abs(deri_theta)) < stop_condition or i >1000:
        break
      else:
      #updating interating times each interation
        self.update(learning_rate)

      #add interating times
      i = i+1

  def other_optimize(self):

    def __optimize_obj(theta):

      z = np.dot(self.__X, theta)
      h = self.sigmoid(z)
      hypos = h.reshape((len(h),1))
      cost = np.dot(-1*self.__Y.transpose(), np.log(hypos))+np.dot(-1*(1-self.__Y).transpose(),np.log(1-hypos))

      return cost

    def __deriv_obj(theta):

      deri_theta = np.zeros_like(self.__theta)
      z = np.dot(self.__X, theta)
      h = self.sigmoid(z)
      hypos = h.reshape((len(h),1))
    #calculate derivative for the features
      for i in range(self.__num_of_feature):

        temp = hypos-self.__Y
        deri_theta_i = (1/len(self.__X)) * np.dot(temp.transpose(), self.__X[:,i])
        deri_theta[i] = deri_theta_i

      return deri_theta

    theta = np.zeros_like(self.__theta)
    res = minimize(__optimize_obj, theta, method='BFGS', jac=__deriv_obj, options={'disp': True})
    print(res.x)

  def get_XY(self):
    return self.__X,self.__Y

  def get_theta(self):
    return self.__theta

  def get_num_feature(self):
    return self.__num_of_feature

feature set up

In [ ]:
start = '2012-12-12'
end = '2013-12-30'
Y = yf.download('AAPL',start=start,end=end,interval='1d')
X = yf.download('^GSPC',start=start,end=end,interval='1d')
X_price = X['Adj Close']
Y_price = Y['Adj Close']
ret_Y = Y_price.pct_change()
ret_Y = ret_Y.fillna(0)
vol_X = X['Volume']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
test_case = ML_logistic(X_price,ret_Y)
test_case.add_column(vol_X)

In [ ]:
test_case.gradient_descent(learning_rate=0.03,stop_condition=0.01)

Cost on interation1 is [[182.29770849]]
[0.11689647]
Cost on interation2 is [[182.25148932]]
[0.1158335]
Cost on interation3 is [[182.20611583]]
[0.11478023]
Cost on interation4 is [[182.16157252]]
[0.1137366]
Cost on interation5 is [[182.11784417]]
[0.11270252]
Cost on interation6 is [[182.0749158]]
[0.11167791]
Cost on interation7 is [[182.03277273]]
[0.1106627]
Cost on interation8 is [[181.99140053]]
[0.10965682]
Cost on interation9 is [[181.95078501]]
[0.10866019]
Cost on interation10 is [[181.91091227]]
[0.10767273]
Cost on interation11 is [[181.87176863]]
[0.10669437]
Cost on interation12 is [[181.83334067]]
[0.10572503]
Cost on interation13 is [[181.79561521]]
[0.10476463]
Cost on interation14 is [[181.7585793]]
[0.10381311]
Cost on interation15 is [[181.72222023]]
[0.10287038]
Cost on interation16 is [[181.68652553]]
[0.10193637]
Cost on interation17 is [[181.65148294]]
[0.10101101]
Cost on interation18 is [[181.61708044]]
[0.10009422]
Cost on interation19 is [[181.5833062]]
[0

In [ ]:
test_case.get_theta()

array([-0.03380907,  0.14722114, -0.18013187])

Other way around to find theta

In [ ]:
test_case.other_optimize()

Optimization terminated successfully.
         Current function value: 179.733782
         Iterations: 10
         Function evaluations: 11
         Gradient evaluations: 11
[-0.03899769  0.15726443 -0.20070565]
